# rpart: Recursive Partitioning and Regression Trees

rpart决策树的特点：

1. 分类和回归的自变量既允许连续取值，也允许离散取值；
1. 不要求自变量之间相互独立；
1. 方便评价自变量的重要性；
1. 方便实现代价敏感分类；
1. 有效地应对自变量缺失。

算法参见：
- https://zhuanlan.zhihu.com/p/85731206
- https://zhuanlan.zhihu.com/p/139519852
- https://zhuanlan.zhihu.com/p/82054400

  注：单个博文的介绍不一定详尽和正确，应综合参考。

# 算法解析


1. 在二维平面上直观地观察rpart决策树分类和回归--可较好地拟合理论曲线；
2. 平衡、代价不敏感、无缺失值、自变量混合取值分类实例：了解cp值对模型复杂度的控制；基于1-SE原则确定最优cp值；画出决策树，了解其清晰的层次判断结构；通过比对最优决策树在训练集和测试集上的误差，学会怎样将得到的决策树应用于预测；了解rpart算法在分类时如何评价(自变量)变量的重要性；学习当目标变量有二个以上类别时，如何判读决策树图。
3. 非平衡、代价敏感、无缺失值分类实例：了解先验法、代价法、权重法的实现方式，发现三者等效。
4. $friedman1$数据集 ，是一个简单和较为普通的分类问题，通过此算例了解决策树分类的基本算法：cp表各项值，特别是xerror的含义；观察xerror随cp降低的变化趋势--通过1-SE原则找到最优cp值，构建最优决策树；通过训练集和测试集错判率的cp曲线验证1-SE最优cp值的准确数，最后讲解变量重要性评价方式。

## 直观观察

### 分类界线

In [ ]:
library(pacman)

options(warn = -1) # 忽略一切警告
options("width" = 140) # 充分利用打印宽度
options(repr.plot.width = 15, repr.plot.height = 10) # 满幅

p_load(rpart,rattle,treemisc,skimr)
s <- skim_tee    #勘察数据

**生成训练集**

In [ ]:
n <- 10000
u <- 10 # 取值范围
v <- 5  # 噪声影响强度

set.seed(11)
x1 <- sort(runif(n, min = -u, max = u))
x2 <- x1 * cos(x1) + v * rnorm(n) - 4
y <- 1
d0 <- data.frame(y, x1, x2)

set.seed(101)
x1 <- sort(runif(n, min = -u, max = u))
x2 <- x1 * cos(x1) + v * rnorm(n) + 4
y <- 2
d1 <- data.frame(y, x1, x2)

d_tr <- rbind(d0, d1)
d_tr$y <- factor(d_tr$y) # 生成训练集
plot(d_tr$x1, d_tr$x2, col = d_tr$y, xlab = "x1", ylab = "x2", cex = 0.5)
x2 <- x1 * cos(x1)
lines(x1, x2, col = "blue", lwd = 6, lty = 3, add = TRUE)
legend("bottomright", "理论分类界限", lwd = 6, col = "blue", lty = 3, cex = 0.8)
grid()

**cp曲线**

In [ ]:
set.seed(2) # 固定xstd：xerror具有随机性--数据随机分割的原因
ct <- rpart(y ~ ., data = d_tr, method = "class", cp = 0)   # cp=0:取消cp在模型复杂度上的约束
plotcp(ct)
grid()

上图曲线与水平虚线相交最左边的点即最优cp值(1-SE规则:最低xerror上浮其一个标准差所对应的最大cp值)

**完全树**--cp=0：决策树充分生长(除了其他默认分枝要求)，可见分类界限复杂

In [ ]:
plot(d_tr$x1, d_tr$x2, col = d_tr$y, xlab = "x1", ylab = "x2", cex = 0.5)

ct1 <- rpart(y ~ ., data = d_tr, method = "class", cp = 0)

x1 <- seq(min(d_tr$x1), max(d_tr$x1), length = 100)
x2 <- seq(min(d_tr$x2), max(d_tr$x2), length = 100)
pd <- function(x1, x2){
  predict(ct1, data.frame(x1, x2), type = "prob")[, 1]
}

z <- outer(x1, x2, FUN = pd)
contour(x1, x2, z,
  levels = 0.5, drawlabels = FALSE,
  lwd = 6, col = "blue", add = TRUE
) # 画出分类界限--置信度为0.5的等高线

legend("bottomright", "分类界限-完全树", lwd = 6, col = "blue", lty = 1,cex = 0.8)
grid()

**最简树**--只有根节点：分类界限极度简单

In [ ]:
plot(d_tr$x1, d_tr$x2, col = d_tr$y, xlab = "x1", ylab = "x2", cex = 0.5)

ct2 <- rpart(y ~ ., data = d_tr, method = "class", cp = Inf)
x1 <- seq(min(d_tr$x1), max(d_tr$x1), length = 100)
x2 <- seq(min(d_tr$x2), max(d_tr$x2), length = 100)

pd <- function(x1, x2){
  predict(ct2, data.frame(x1, x2), type = "prob")[, 1]
}

z <- outer(x1, x2, FUN = pd)
contour(x1, x2, z,
  levels = 0.5, drawlabels = FALSE,
  lwd = 6, col = "blue", add = TRUE
) # 画出分类界限--置信度为0.5的等高线

x2 <- x1 * cos(x1)
lines(x1, x2,col = "blue",lwd = 6, lty = 3)
legend("bottomright", c("分类界限-理论"), lwd = 6, col = "blue", lty = 3, cex = 0.8)
grid()
#此时决策树只有一个根节点--将所有的样本点混为一谈,所以没有分界线

**最优树**--xerror尽量低且树尽量小：接近理论分类界限

In [ ]:
plot(d_tr$x1, d_tr$x2, col = d_tr$y, xlab = "x1", ylab = "x2", cex = 0.5)


cto <- prune_se(ct1, prune = TRUE, se = 1)      #1SE最优树
x1 <- seq(min(d_tr$x1), max(d_tr$x1), length = 100)
x2 <- seq(min(d_tr$x2), max(d_tr$x2), length = 100)

pd <- function(x1, x2){
  predict(cto, data.frame(x1, x2), type = "prob")[, 1]
}

z <- outer(x1, x2, FUN = pd)
contour(x1, x2, z,
  levels = 0.5, drawlabels = FALSE,
  lwd = 6, col = "blue", add = TRUE
) # 画出分类界限--置信度为0.5的等高线

x2 <- x1 * cos(x1)
lines(x1, x2,col = "blue",lwd = 6, lty = 3)
legend("bottomright", c("分类界限-理论", "分类界限-最优决策树"), lwd = c(6, 6), col = c("blue", "blue"), lty = c(3, 1),cex = 0.8)
grid()

**cp 对分类性能的影响**

In [ ]:
n <- 10000
u <- 10
v <- 5

set.seed(10)
x1 <- sort(runif(n, min = -u, max = u))
x2 <- x1 * cos(x1) + v * rnorm(n) - 4
y <- 1
d0 <- data.frame(y, x1, x2)

set.seed(100)
x1 <- sort(runif(n, min = -u, max = u))
x2 <- x1 * cos(x1) + v * rnorm(n) + 4
y <- 2
d1 <- data.frame(y, x1, x2)

d_te <- rbind(d0, d1)    
d_te$y <- factor(d_te$y)    #生成测试集

ct <- rpart(y ~ ., data = d_tr, method = "class", cp = 0)
cp <- ct$cp
a <- cp[, 1] # cp序列
L <- length(a)
Cp <- sqrt(a[-L] * a[-1]) # 几何平均值--二个界值之间的值
node <- cp[-1, 2] + 1 # 叶节点个数序列

error_train <- c()
error_test <- c()

for (c in Cp) # 依次画出各个cp下的决策树，计算当时的训练集和测试集错误率
{
  ct <- rpart(y ~ ., data = d_tr, method = "class", cp = c)
  # rpart.plot(twf,main=paste('CP=',round(c,4)))    #当前cp值对应的决策树

  pred_tr <- predict(ct, d_tr, type = "class")
  error_train <- c(error_train, mean(pred_tr != d_tr$y))

  pred_te <- predict(ct, d_te, type = "class")
  error_test <- c(error_test, mean(pred_te != d_te$y))
}

p_load(plotly)

fig <- plot_ly(x = Cp) %>%
  add_trace(y = error_train, type = "scatter", name = "train set", color = I("black"), mode = "lines+markers") %>%
  add_trace(y = error_test, type = "scatter", name = "test  set", color = I("red"), mode = "lines+markers") %>%
  layout(
    title = "Error curves of train and test set",
    xaxis = list(title = "cp"),
    yaxis = list(title = "error")
  )
fig


cat("\n\n")

fig <- plot_ly(x = node) %>%
  add_trace(y = error_train, type = "scatter", name = "train set", color = I("black"), mode = "lines+markers") %>%
  add_trace(y = error_test, type = "scatter", name = "test  set", color = I("red"), mode = "lines+markers") %>%
  layout(
    title = "Error curves of train and test set",
    xaxis = list(title = "number of leaf node"),
    yaxis = list(title = "error")
  )
fig
#可见由xerror得到的最优cp和叶节点个数较为准确 

### 回归曲线

In [ ]:
library(pacman)

options(warn = -1) # 忽略一切警告
options("width" = 140) # 充分利用打印宽度
options(repr.plot.width = 15, repr.plot.height = 10) # 满幅

p_load(rpart,rattle,treemisc,skimr)
s <- skim_tee

**构建训练集**

In [ ]:
n <- 10000
u <- 10
v <- 5

set.seed(101)
x <- sort(runif(n, min = -u, max = u))
y <- x * cos(x) + v * rnorm(n) 
plot(x, y, cex = 0.5)

d_tr <- data.frame(x, y)

**cp曲线**

In [ ]:
set.seed(2) # 固定xstd：xerror具有随机性--数据随机分割的原因
rt <- rpart(y ~ ., data = d_tr, cp = 0)   # y为连续变量，rpart函数自动做回归计算
plotcp(rt)
grid()

**完全树**--cp=0：回归曲线复杂

In [ ]:
rt1 <- rpart(y ~ ., data = d_tr, cp = 1e-5)
x <- seq(min(d_tr$x), max(d_tr$x), length = 500)

plot(d_tr$x, d_tr$y, xlab = "x", ylab = "y")
lines(d_tr$x, predict(rt1, d_tr), type = "l", lwd = 1, col = "blue")
y <- x * cos(x)
lines(x, y, type = "l", lwd = 4, xlab = "x", ylab = "y", col = "red")
legend("bottomright", c("回归曲线--理论", "回归曲线--完全树"), lwd = c(4, 1), col = c("red", "blue"), cex = 0.8)
grid()

**最简树**--只有根节点：回归曲线为一条水平线

In [ ]:
rt2 <- rpart(y ~ ., data = d_tr, cp = Inf)
x <- seq(min(d_tr$x), max(d_tr$x), length = 500)

plot(d_tr$x, d_tr$y, xlab = "x", ylab = "y")
lines(d_tr$x, predict(rt2, d_tr), type = "l", lwd = 6, col = "blue")
y <- x * cos(x)
lines(x, y, type = "l", lwd = 4, xlab = "x", ylab = "y", col = "red")
legend("bottomright", c("回归曲线--理论", "回归曲线--最简树"), lwd = c(4, 4), col = c("red", "blue"), cex = 0.8)
grid()

**最优树**--xerror尽量低且决策树尽量简单：接近理论回归曲线

In [ ]:
rto <- prune_se(rt1, prune = TRUE, se = 1)
x <- seq(min(d_tr$x), max(d_tr$x), length = 500)

plot(d_tr$x, d_tr$y, xlab = "x", ylab = "y")
lines(d_tr$x, predict(rto, d_tr), type = "l", lwd = 6, col = "blue")
y <- x * cos(x)
lines(x, y, type = "l", lwd = 4, xlab = "x", ylab = "y", col = "red")
legend("bottomright", c("回归曲线--理论", "回归曲线--最优树"), lwd = c(4, 4), col = c("red", "blue"), cex = 0.8)
grid()

**cp对回归性能的影响**

In [ ]:
n <- 10000
u <- 10
v <- 5

set.seed(111)
x <- sort(runif(n, min = -u, max = u))
y <- x * cos(x) + v * rnorm(n) 
d_te <- data.frame(x, y) #生成测试集

rt <- rpart(y ~ ., data = d_tr, cp = 0) 
cp <- rt$cp
a <- cp[, 1] # cp序列
L <- length(a)
Cp <- sqrt(a[-L] * a[-1]) # 几何平均值--二个界值之间的值
node <- cp[-1, 2] + 1 # 叶节点个数序列

error_train <- c()
error_test <- c()

for (c in Cp) # 依次画出各个cp下的决策树，计算当时的训练集和测试集错误率
{
  rt <- rpart(y ~ ., data = d_tr, cp = c)
  # rpart.plot(twf,main=paste('CP=',round(c,4)))    #当前cp值对应的决策树

  pred_tr <- predict(rt, d_tr)
  error_train <- c(error_train, sqrt(mean((pred_tr - d_tr$y)^2)))

  pred_te <- predict(rt, d_te)
  error_test <- c(error_test, sqrt(mean((pred_te - d_te$y)^2)))
}

p_load(plotly)

fig <- plot_ly(x = Cp) %>%
  add_trace(y = error_train, type = "scatter", name = "train set", color = I("black"), mode = "lines+markers") %>%
  add_trace(y = error_test, type = "scatter", name = "test  set", color = I("red"), mode = "lines+markers") %>%
  layout(
    title = "Error curves of train and test set",
    xaxis = list(title = "cp"),
    yaxis = list(title = "error")
  )
fig

cat("\n\n")

fig <- plot_ly(x = node) %>%
  add_trace(y = error_train, type = "scatter", name = "train set", color = I("black"), mode = "lines+markers") %>%
  add_trace(y = error_test, type = "scatter", name = "test  set", color = I("red"), mode = "lines+markers") %>%
  layout(
    title = "Error curves of train and test set",
    xaxis = list(title = "number of leaf node"),
    yaxis = list(title = "error")
  )
fig
#可见由xerror得到的最优cp和叶节点个数较为准确 

## 分类
### 代价不敏感

abalone数据集来自4177只鲍鱼，自变量包括type(公、母、幼)、LongestShell 、Diameter、 Height和几种重量(WholeWeight、ShuckedWeight、VisceraWeightShellWeight)，因变量是Rings(外壳上的环数)。

将abalone 数据集的Rings值改造成"L"、 "H"二个数量大致相同的类别，当前的问题是根据一系列自变量，判断abalone 的Rings类别，自变量混合取值--**既有离散取值，也有连续取值**。


In [ ]:
library(pacman)

options(warn = -1) # 忽略一切警告
options("width" = 140) # 充分利用打印宽度
options(repr.plot.width = 15, repr.plot.height = 10) # 满幅

p_load(rpart)
p_load(skimr)
p_load(partykit)
p_load(AppliedPredictiveModeling)
s <- skim_tee

**导入、改造、勘察数据**

In [ ]:
data(abalone)
ab <- abalone
ab$Rings <- factor((ab$Rings) > 9, labels = c("L", "H"))
# 处理成二类，逻辑型factor的levels的排序默认是：FALSE,TRUE，分别对应于"L","H"
s(ab)

可见Rings的L类和H类数量平衡

**随机平分为训练集和测试集**

In [ ]:
set.seed(100)
i <- sample(nrow(ab), nrow(ab) * 0.5)
tr_ab <- ab[i, ]
te_ab <- ab[-i, ]
s(tr_ab)

In [ ]:
set.seed(2) # 固定xstd：xerror具有随机性--数据随机分割的原因
ctab <- rpart(Rings ~ ., data = tr_ab, cp = 0) # cp=0--取消在模型复杂度上的约束；此时Ring为factor，rpart函数自动采用分类算法
summary(ctab)

**观察以上输出**(点开scrollable element)：



--cp 表：

- rel error 是训练集误差--错判率 = 判错总个数 / 样例总个数。
  - 随着 cp 的下降，nsplit 全程上升(决策树越来越庞大)，rel error 全程而下降，表明 cp 越低决策树越复杂，训练集误差越低。
- xerror 是多折(默认为十折)交叉验证得到的误差值均值。
  - xerror 首先随着随着 cp 的下降而下降，然后开始上升，表明 cp 越低决策树越复杂，会引起过拟合，降低泛化性能。
  - xerror 都大于同行的 rel error，可见 rel error 偏乐观。
- rel error 和 xerror 都是以 Root node error 为基准的相对值--为了便于比较。
- xstd 是 xerror 的标准差，反映 xerror 的变动情况，xerror 具有随机性--数据随机分割的缘故。
  - 1-SE 原则：最优 cp 值为最小 xerror 的 cp 点的 1-SE 范围内的最大值，参见 https://stats.stackexchange.com/questions/92547/r-rpart-cross-validation-and-1-se-rule-why-is-the-column-in-cptable-called-xst 

--各splits(分枝)的improve值逐层下降--优先选择improve大的变量作为当前的分枝变量。

**cp曲线--cp 逐降，xerror 的变化情况：**


In [ ]:
plotcp(ctab) # 图示，横坐标值是上表二个连续cp值的几何平均值--取值可以介于二者之间之意
grid()
# 水平虚线为xerror最小均值加其1-SE，在此线附近最大的cp值最优，可见cp在0.0045附近，xerror小，根据1-SE原则，0.011附近cp最优

**最优决策树**

In [ ]:
ct_abO <- prune_se(ctab, prune = TRUE, se = 1)    #1-SE最优树
plot(as.party(ct_abO))

In [ ]:
pred_tr <- predict(ct_abO, tr_ab, type = "class") # 训练集的预测值
(error_train <- mean(pred_tr != tr_ab$Rings)) # 训练集的错误率
pred_te <- predict(ct_abO, te_ab, type = "class") # 测试集的预测值
(error_test <- mean(pred_te != te_ab$Rings)) # 测试集的错误率

**变量的重要性排序**

变量可以作为主变量或代理变量多次出现在树中。**变量重要性的总体度量是将其作为主要变量的每个分割的分割度量的优度(improve)之和，加上将其作为替代变量的所有分割的优度(improve)_(adjusted agreement)**。打印输出时，这些值被缩放到总和为100，并显示四舍五入的值，省略任何比例小于 1%的变量。

In [ ]:
barplot(ctab$variable.importance)

--“improve”怎么计算？

The improvement is n times the change in impurity index

improvement = 分枝前(节点样例个数 × “impurity index”) - 分支后(节点样例个数 × “impurity index”)之和

比如根节点的分枝：


In [ ]:
addmargins(with(
  tr_ab,
  table(cut(ShellWeight, c(0, 0.2545, Inf)),
  Rings, exclude = NULL)
))

improvement = 1044 - 1166[ 1-(851/1166)<sup>2</sup> - (315/1166)<sup>2</sup> ] - 922[ 1 - (193/922)<sup>2</sup>-(729/922)<sup>2</sup> ] = 278.9977


--“Surrogate splits”怎么得到？

按照“agree”降序排列，“agree”是该替代分枝与原分枝的一致程度，以根节点的分枝为例：

In [ ]:
with(tr_ab, table(cut(ShellWeight, c(0, 0.2545, Inf)), cut(WholeWeight, c(0, 0.89975, Inf)), exclude = NULL))

注意：原分枝是“ShellWeight < 0.2545 to the left”，替代分枝是“WholeWeight < 0.89975 to the left”，所以其 agree 为：


In [ ]:
(1100 + 829) / (1100 + 829 + 93 + 66)

“adj”(adjusted agreement)计算：

若该数据集所有的样例的 ShellWeight 值缺失，不依赖其他属性，将其认定为“< 0.2545”,正确的个数为 1100+66，替代分枝“WholeWeight < 0.89975 ”的作用体现在剩下的(1100 + 829 + 93 + 66 - (1100 + 66))样例中正确认定了(1100 + 829 - (1100 + 66))个，所以 adj 为：

In [ ]:
(1100 + 829 - (1100 + 66)) / (1100 + 829 + 93 + 66 - (1100 + 66))

目标变量有二个以上类别：

In [ ]:
library(pacman)

options(warn = -1) # 忽略一切警告
options("width" = 140) # 充分利用打印宽度
options(repr.plot.width = 15, repr.plot.height = 10) # 满幅

p_load(rpart,skimr,partykit)
s <- skim_tee

In [ ]:
s(iris)

In [ ]:
plot(as.party(prune_se(rpart(Species ~ ., data = iris), prune = TRUE, se = 1)))

### 代价敏感

**rpart针对代价敏感问题的三种算法：先验法、代价法、权重法**



将abalone数据集改造成非平衡(代价敏感问题常出现于非平衡数据集)，假定将Rings的“H”类判为“L”代价为5，将“L”类判为“H”代价为1

In [ ]:
library(pacman)

options(warn = -1) # 忽略一切警告
options("width" = 140) # 充分利用打印宽度
options(repr.plot.width = 15, repr.plot.height = 10) # 满幅

p_load(rpart, AppliedPredictiveModeling, skimr, treemisc, partykit)
s <- skim_tee

**导入、改造、勘察数据**

In [ ]:
data(abalone)
ab <- abalone
ab$Rings <- factor((ab$Rings) > 12, labels = c("L", "H"))
s(ab)

可见目标变量Rings的类别并不平衡，数量L类约为H类的5倍

**将数据随机平分为训练集和测试集：**

In [ ]:
set.seed(100)
i <- sample(nrow(ab), nrow(ab) * 0.5)
tr_ab <- ab[i, ]
te_ab <- ab[-i, ]
s(tr_ab)

In [ ]:
(ct <- rpart.control(cp = 1e-5))    #降低cp值，使得决策树能够充分生长，但奇怪的是若将cp置为0，反而三种方法得到的决策树不完全一样

#### 先验法

In [ ]:
set.seed(22)
(levels(tr_ab$Rings))     #先验概率的类别排列顺序
(pr <- c(1737 / 2088, (1 - 1737 / 2088) * 5) / sum(c(1737 / 2088, (1 - 1737 / 2088) * 5))) # 设定先验概率
(ct_ab1 <- rpart(Rings ~ ., data = tr_ab, parms = list(prior = pr), control = ct))

#### 代价法

In [ ]:
set.seed(22)
(levels(tr_ab$Rings))     #代价矩阵中行和列中的类别排列顺序
(lm <- matrix(c(0, 1, 5, 0), byrow = TRUE, nrow = 2)) # L类判为H类的代价为1，H类判为L类的代价为5
(ct_ab2 <- rpart(Rings ~ ., data = tr_ab, parms = list(loss = lm), control = ct))

#### 权重法

In [ ]:
w <- ifelse(tr_ab$Rings == "H", 5, 1)
set.seed(22)
(ct_ab3 <- rpart(Rings ~ ., data = tr_ab, weights = w, control = ct))


#### 结论

1. 先验法、代价法、权重法得到的决策树完全相同，三种等效
2. 在表达上，权重法最为简单、清晰
   
注：可以通过https://www.diffchecker.com/zh-Hans/text-compare/ ，比较三者

## 回归

### 自变量混合取值

In [ ]:
library(pacman)

options(warn = -1) # 忽略一切警告
options("width" = 140) # 充分利用打印宽度
options(repr.plot.width = 15, repr.plot.height = 10) # 满幅

p_load(rpart)
p_load(rattle) #使用其中的fancyRpartPlot函数
p_load(skimr)
p_load(treemisc)
s <- skim_tee
p_load(AppliedPredictiveModeling)

In [ ]:
data(abalone)
ab <- abalone
s(ab)

In [ ]:
# 将数据集1:1随机分割成训练集和测试集
set.seed(100)
I <- sample(nrow(ab), nrow(ab) * 0.5)
tr_ab <- ab[I, ]
te_ab <- ab[-I, ]
s(tr_ab)

In [ ]:
set.seed(2) # 固定xstd：xerror具有随机性--数据随机分割的原因
rt_ab <- rpart(Rings ~ ., data = tr_ab, cp = 0) # cp=0--决策树充分生长，保留该值以上cp的情形
summary(rt_ab)

- improve is of a split is SST − (SSL + SSR), SST is the sum of square errors for the node, and SSR, SSL are the sums of square errors for the right and left son
- The error of a node is the variance of y

In [ ]:
rt_ab$method   #method取值

**cp曲线**

In [ ]:
plotcp(rt_ab) 
grid()

**xerror 1-SE最优树**


In [ ]:
rt_abO <- prune_se(rt_ab, prune = TRUE, se = 1)    
fancyRpartPlot(rt_abO, caption = "")

In [ ]:
plot(as.party(rt_abO))   #叶节点上直接反映因变量值的分布

**误差－最优树**


In [ ]:
pred_tr <- predict(rt_abO, tr_ab) # 训练集的预测值
(error_train <- sqrt(mean((pred_tr - tr_ab$Rings)^2))) # 针对于训练集的RMSE
pred_te <- predict(rt_abO, te_ab) # 测试集的预测值
(error_test <- sqrt(mean((pred_te - te_ab$Rings)^2))) # 针对于训练集的RMSE

**cp 对回归性能的影响**


In [ ]:
cp <- rt_ab$cp
a <- cp[, 1] # cp序列
L <- length(a)
Cp <- sqrt(a[-L] * a[-1]) # 几何平均值--二个界值之间的值
node <- cp[-1, 2] + 1 # 叶节点个数序列

error_train <- c()
error_test <- c()

for (c in Cp) # 依次画出各个cp下的决策树，计算当时的训练集和测试集错误率
{
  rt <- rpart(Rings ~ ., data = tr_ab, cp = c)
  # rpart.plot(twf,main=paste('CP=',round(c,4)))    #当前cp值对应的决策树

  pred_tr <- predict(rt, tr_ab)
  error_train <- c(error_train, sqrt(mean((pred_tr - tr_ab$Rings)^2)))

  pred_te <- predict(rt, te_ab)
  error_test <- c(error_test, sqrt(mean((pred_te - te_ab$Rings)^2)))
}

p_load(plotly)

fig <- plot_ly(x = Cp) %>%
  add_trace(y = error_train, type = "scatter", name = "train set", color = I("black"), mode = "lines+markers") %>%
  add_trace(y = error_test, type = "scatter", name = "test  set", color = I("red"), mode = "lines+markers") %>%
  layout(
    title = "Error curves of train and test set",
    xaxis = list(title = "cp"),
    yaxis = list(title = "error")
  )
fig

cat("\n\n")

fig <- plot_ly(x = node) %>%
  add_trace(y = error_train, type = "scatter", name = "train set", color = I("black"), mode = "lines+markers") %>%
  add_trace(y = error_test, type = "scatter", name = "test  set", color = I("red"), mode = "lines+markers") %>%
  layout(
    title = "Error curves of train and test set",
    xaxis = list(title = "number of leaf node"),
    yaxis = list(title = "error")
  )
fig

### 自变量连续取值


数据集 $friedman1$按此方式生成模拟数据：自变量 $\{X_j\}^{10}_ {j=1}\overset{iid}{\sim} U(0,1)$，$Y=10\sin(\pi X_1X_2)+20(X_3-0.5)^2+10X_4+5X_5+\epsilon$ ,  其中$\epsilon\sim N(0,\sigma)$ ，请注意$X_6 \sim X_{10} $ 独立于 $Y$ 。

In [ ]:
library(pacman)

options(warn = -1) # 忽略一切警告
options("width" = 140) # 充分利用打印宽度
options(repr.plot.width = 15, repr.plot.height = 10) # 满幅

p_load(rpart, skimr, treemisc, partykit, rattle)
s <- skim_tee

In [ ]:
set.seed(943) # for reproducibility
fr <- treemisc::gen_friedman1(1000, nx = 7, sigma = 0.1)
s(fr)

In [ ]:
# 将数据集1:1随机分割成训练集和测试集
set.seed(100)
I <- sample(nrow(fr), nrow(fr) * 0.5)
tr_fr <- fr[I, ]
te_fr <- fr[-I, ]
s(tr_fr)

In [ ]:
set.seed(2) # 固定xstd：xerror具有随机性--数据随机分割的原因
rt_fr <- rpart(y ~ ., data = tr_fr, cp = 0) # cp=0--决策树充分生长，保留该值以上cp的情形
plotcp(rt_fr) # 随着cp的变化，决策树的error变化情况
grid()

**xerror 1-SE 最优树**


In [ ]:
rt_fro <- prune_se(rt_fr, prune = TRUE, se = 1)    
fancyRpartPlot(rt_fro, caption = "")

**自变量重要的排序**

In [ ]:
barplot(rt_fro$variable.importance)

实际上X6、X7与因变量无关，此排序未能正确反映此关系

**错误率－最优树**


In [ ]:
pred_tr <- predict(rt_fro, tr_fr) # 训练集的预测值
(MSE_train <- mean((pred_tr - tr_fr$y)^2)) # 针对于训练集的MSE
pred_te <- predict(rt_fro, te_fr) # 测试集的预测值
(MSE_test <- mean((pred_te - te_fr$y)^2)) # 针对于训练集的MSE

## Poisson 回归：将计数值作为回归的目标


理论参见：

- https://zhuanlan.zhihu.com/p/104494467
- https://mengte.online/archives/4373


In [ ]:
library(pacman)

options(warn = -1) # 忽略一切警告
options("width" = 140) # 充分利用打印宽度
options(repr.plot.width = 15, repr.plot.height = 10) # 满幅

p_load(rpart)
p_load(rattle) # 使用其中的fancyRpartPlot函数
p_load(skimr)
s <- skim_tee

In [ ]:
s(solder.balance)

In [ ]:
sfit <- rpart(skips ~ Opening + Solder + Mask + PadType + Panel, data = solder.balance, method = "poisson")

In [ ]:
summary(sfit)

improve--The likelihood ratio test for two Poisson groups: D<sub>parent</sub>- (D<sub>left</sub> + D<sub>right</sub>)


In [ ]:
plotcp(sfit)

In [ ]:
sfito <- rpart(skips ~ Opening + Solder + Mask + PadType + Panel,
    data = solder.balance,
    method = "poisson", cp = 0.017
)

In [ ]:
fancyRpartPlot(sfito)

### 生存分析：一类特殊的 Possion 回归


理论参见：

- https://www.cnblogs.com/wwxbi/p/6136348.html
- http://thisis.yorven.site/blog/index.php/2020/04/06/survival-analysis/


In [ ]:
library(pacman)

options(warn = -1) # 忽略一切警告
options("width" = 140) # 充分利用打印宽度
options(repr.plot.width = 15, repr.plot.height = 10) # 满幅
p_load(rpart.plot)
p_load(rpart)
p_load(skimr)
s <- skim_tee

In [ ]:
stagec

In [ ]:
s(stagec)

In [ ]:
pfit <- rpart(survival::Surv(pgtime, pgstat) ~ age + eet + g2 + grade + gleason + ploidy, data = stagec)
summary(pfit)

In [ ]:
pfit$method

Sur 函数参数：

- time：for right censored data, this is the follow up time. For interval data, the first argument is the starting time for the interval.
- event：The status indicator, normally 0=alive, 1=dead.


In [ ]:
plotcp(pfit)
grid()

In [ ]:
pfit2 <- prune(pfit, cp = 0.12945955)
rpart.plot(pfit2, nn = TRUE)
# 各节点第一行、第二行数字的含义是？

In [ ]:
train <- data.frame(RearEnd = c(TRUE, FALSE, TRUE), Fraud = c(TRUE, FALSE, TRUE))
mytree <- rpart(Fraud ~ RearEnd, data = train, method = "class", minsplit = 2, minbucket = 1)
rpart.plot(mytree, extra = 104)
mytree$frame

In [ ]:
train <- data.frame(RearEnd = c(TRUE, FALSE, TRUE), Fraud = c(TRUE, FALSE, TRUE))
mytree <- rpart(Fraud ~ RearEnd,
    data = train, method = "class", minsplit = 2, minbucket = 1,
    weights = c(0.1, 0.1, 0.1)
)
rpart.plot(mytree, extra = 104)
mytree$frame

In [ ]:
train <- data.frame(RearEnd = c(TRUE, FALSE, TRUE), Fraud = c(TRUE, FALSE, TRUE))
mytree <- rpart(Fraud ~ RearEnd,
    data = train, method = "class", minsplit = 2, minbucket = 1,
    weights = c(0.4, 0.4, 0.4)
)
rpart.plot(mytree, extra = 104)
mytree$frame

In [ ]:
fancyRpartPlot(mytree, caption = "")

## 缺失值处理


### 建模时自变量值缺失


In [ ]:
library(pacman)

options(warn = -1) # 忽略一切警告
options("width" = 140) # 充分利用打印宽度
options(repr.plot.width = 15, repr.plot.height = 10) # 满幅

p_load(rpart)
p_load(rattle) # 使用其中的fancyRpartPlot函数
p_load(skimr)
s <- skim_tee
p_load(AppliedPredictiveModeling)

In [ ]:
data(abalone)
ab <- abalone
ab$Rings <- factor((ab$Rings) > 9, labels = c("L", "H")) # 处理成二类，逻辑型factor的levels的排序默认是：FALSE,TRUE，分别对应于"L","H"
s(ab)

In [ ]:
# 将数据集1:1随机分割成训练集和测试集
set.seed(100)
I <- sample(nrow(ab), nrow(ab) * 0.5)
tr_ab <- ab[I, ]
te_ab <- ab[-I, ]
s(tr_ab)

将训练集中第一行的ShellWeight值置为缺失

In [ ]:
tr_ab[1, "ShellWeight"] <- NA
s(tr_ab)

In [ ]:
with(tr_ab, table(Rings, exclude = NULL))

In [ ]:
with(tr_ab, table(cut(ShellWeight, c(0, 0.2545, Inf)), Rings, exclude = NULL))

In [ ]:
set.seed(2) 
ctab <- rpart(Rings ~ ., data = tr_ab, method = "class", cp = 0) 
summary(ctab)

 可见：

 "Primary splits:
      ShellWeight   < 0.2545  to the left,  improve=279.7491, (1 missing)"---有一个值缺失
      
 ...

 "Surrogate splits:
      WholeWeight   < 1.339   to the left,  agree=0.868, adj=0.627, (1 split)"---使用WholeWeight替代作了分枝

但是：

 "Primary splits:
      ShellWeight   < 0.2545  to the left,  improve=279.7491, (1 missing)"---279.7491不知道怎么计算出来的

### 预测时自变量值缺失


In [ ]:
library(pacman)

options(warn = -1) # 忽略一切警告
options("width" = 140) # 充分利用打印宽度
options(repr.plot.width = 15, repr.plot.height = 10) # 满幅

p_load(rpart)
p_load(rattle) # 使用其中的fancyRpartPlot函数
p_load(skimr)
s <- skim_tee
p_load(AppliedPredictiveModeling)

In [ ]:
data(abalone)
ab <- abalone
ab$Rings <- factor((ab$Rings) > 9, labels = c("L", "H")) # 处理成二类，逻辑型factor的levels的排序默认是：FALSE,TRUE，分别对应于"L","H"

# 将数据集1:1随机分割成训练集和测试集
set.seed(100)
I <- sample(nrow(ab), nrow(ab) * 0.5)
tr_ab <- ab[I, ]
te_ab <- ab[-I, ]
head(te_ab)

In [ ]:
set.seed(2) # 固定xstd：xerror具有随机性--数据随机分割的原因
ctab <- rpart(Rings ~ ., data = tr_ab, control = list(maxsurrogate = 7), method = "class", cp = 0.2)
summary(ctab)
fancyRpartPlot(ctab, caption = "")

In [ ]:
#生成该树的控制参数
ctab$control

maxsurrogate--决策树中保留的代理分枝的数量--可能该代理分枝变量值也缺失，则用下一个。如果将其设置为零，计算时间将减少，因为大约一半的计算时间(除了设置)用于搜索代理分割。

usesurrogate--如何在分枝过程中使用代理。0表示只显示；缺少主分割规则值的观察值不会被发送到树的下一级。1表示使用按顺序代理分枝，如果所有的代理缺失，则观察结果不分裂。对于值2，如果缺少所有的代理，则以多数为准返回结果。


**观察“Surrogate splits”的用法**


首先，“ShellWeight”值缺失：


In [ ]:
te_ab[1, "ShellWeight"] <- NA
te_ab[1, ]

In [ ]:
predict(ctab, te_ab[1, ], type = "class")

In [ ]:
te_ab[16, "ShellWeight"] <- NA
te_ab[16, ]

In [ ]:
predict(ctab, te_ab[16, ], type = "class")

ShellWeight 和 WholeWeight 均缺失


In [ ]:
te_ab[1, "WholeWeight"] <- NA
te_ab[1, ]

In [ ]:
predict(ctab, te_ab[1, ], type = "class")

In [ ]:
te_ab[16, "WholeWeight"] <- NA
te_ab[16, ]

In [ ]:
predict(ctab, te_ab[16, ], type = "class")

ShellWeight 和 WholeWeight、Diameter 均缺失


In [ ]:
te_ab[1, "Diameter"] <- NA
te_ab[1, ]

In [ ]:
predict(ctab, te_ab[1, ], type = "class")

In [ ]:
te_ab[16, "Diameter"] <- NA
te_ab[16, ]

In [ ]:
predict(ctab, te_ab[16, ], type = "class")

**按序使用“Surrogate”时的误差**

      ShellWeight-> WholeWeight-> Diameter-> LongestShell-> VisceraWeight-> ShuckedWeight-> Height->Type 依次缺失


In [ ]:
# 无缺失
te_ab <- ab[-I, ]
s(te_ab)

pred_te <- predict(ctab, te_ab, type = "class") # 测试集的预测值
(error_test <- mean(pred_te != te_ab$Rings)) # 测试集的错误率

In [ ]:
te_ab[, "ShellWeight"] <- NA
te_ab[, "ShellWeight"] <- as.numeric(te_ab[, "ShellWeight"])
s(te_ab)

pred_te <- predict(ctab, te_ab, type = "class") # 测试集的预测值
(error_test <- mean(pred_te != te_ab$Rings)) # 测试集的错误率

In [ ]:
te_ab[, "WholeWeight"] <- NA
te_ab[, "WholeWeight"] <- as.numeric(te_ab[, "WholeWeight"])
s(te_ab)

pred_te <- predict(ctab, te_ab, type = "class") # 测试集的预测值
(error_test <- mean(pred_te != te_ab$Rings)) # 测试集的错误率

In [ ]:
te_ab[, "Diameter"] <- NA
te_ab[, "Diameter"] <- as.numeric(te_ab[, "Diameter"])
s(te_ab)

pred_te <- predict(ctab, te_ab, type = "class") # 测试集的预测值
(error_test <- mean(pred_te != te_ab$Rings)) # 测试集的错误率

In [ ]:
te_ab[, "LongestShell"] <- NA
te_ab[, "LongestShell"] <- as.numeric(te_ab[, "LongestShell"])
s(te_ab)

pred_te <- predict(ctab, te_ab, type = "class") # 测试集的预测值
(error_test <- mean(pred_te != te_ab$Rings)) # 测试集的错误率

In [ ]:
te_ab[, "VisceraWeight"] <- NA
te_ab[, "VisceraWeight"] <- as.numeric(te_ab[, "VisceraWeight"])
s(te_ab)

pred_te <- predict(ctab, te_ab, type = "class") # 测试集的预测值
(error_test <- mean(pred_te != te_ab$Rings)) # 测试集的错误率

In [ ]:
te_ab[, "ShuckedWeight"] <- NA
te_ab[, "ShuckedWeight"] <- as.numeric(te_ab[, "ShuckedWeight"])
s(te_ab)

pred_te <- predict(ctab, te_ab, type = "class") # 测试集的预测值
(error_test <- mean(pred_te != te_ab$Rings)) # 测试集的错误率

In [ ]:
te_ab[, "Height"] <- NA
te_ab[, "Height"] <- as.numeric(te_ab[, "Height"])
s(te_ab)

pred_te <- predict(ctab, te_ab, type = "class") # 测试集的预测值
(error_test <- mean(pred_te != te_ab$Rings)) # 测试集的错误率

In [ ]:
te_ab[, "Type"] <- NA
te_ab[, "Type"] <- as.character(te_ab[, "Type"])
s(te_ab)

pred_te <- predict(ctab, te_ab, type = "class") # 测试集的预测值
(error_test <- mean(pred_te != te_ab$Rings)) # 测试集的错误率

In [ ]:
# 所有surrogate缺失，取大类
1037 / (1052 + 1037)

## 时间复杂度


#### classification-自变量全为类别值


In [ ]:
options(warn = -1) # 忽略一切警告
options("width" = 140) # 充分利用打印宽度
options(repr.plot.width = 15, repr.plot.height = 10) # 满幅
p_load(rpart)
p_load(skimr)
s <- skim_tee

In [ ]:
mr <- treemisc::mushroom
s(mr)

In [ ]:
mr <- treemisc::mushroom

N <- 10 # 数据规格数
M <- 100 # 每种数据量实验次数
rc <- matrix(-1, M, N) # 用于存储实验结果
colnames(rc) <- as.character(1:N) # 每列对应于不同的数据规格
nr <- nrow(mr)

pb <- txtProgressBar(style = 3)
for (n in 1:N) {
  for (m in 1:M) {
    set.seed(n * 1000 + m)
    I <- sample(nr, (n * nr / N))
    d <- mr[I, ]
    timestart <- Sys.time()
    rpart(Edibility ~ ., data = d, cp = 0)
    rc[m, n] <- difftime(Sys.time(), timestart, units = "secs")
  }
  setTxtProgressBar(pb, n / N)
}

boxplot(rc, xlab = "Each scale unit represents 10% examples", ylab = "time")
lines(colMeans(rc), type = "o", lwd = 2)
grid()

# 可见呈直线，时间复杂度较低

#### classification-自变量全为连续值


In [ ]:
options(warn = -1) # 忽略一切警告
options("width" = 140) # 充分利用打印宽度
options(repr.plot.width = 15, repr.plot.height = 10) # 满幅
p_load(rpart)
p_load(skimr)
s <- skim_tee

In [ ]:
set.seed(943) # for reproducibility
fr <- treemisc::gen_friedman1(1000, nx = 20, sigma = 0.1)
fr$y <- factor(ifelse(fr$y < 14.2, 0, 1))
s(fr)

In [ ]:
N <- 10 # 数据规格数
M <- 100 # 每种数据量实验次数
rc <- matrix(-1, M, N) # 用于存储实验结果
colnames(rc) <- as.character(1:N) # 每列对应于不同的数据规格
nr <- nrow(fr)

pb <- txtProgressBar(style = 3)
for (n in 1:N) {
  for (m in 1:M) {
    set.seed(n * 1000 + m)
    I <- sample(nr, (n * nr / N))
    d <- fr[I, ]
    timestart <- Sys.time()
    rpart(y ~ ., data = d, cp = 0)
    rc[m, n] <- difftime(Sys.time(), timestart, units = "secs")
  }
  setTxtProgressBar(pb, n / N)
}

boxplot(rc, xlab = "Each scale unit represents 10% examples", ylab = "time")
lines(colMeans(rc), type = "o", lwd = 2)
grid()

# 可见呈直线，时间复杂度较低

#### regression-自变量全为连续值


In [ ]:
options(warn = -1) # 忽略一切警告
options("width" = 140) # 充分利用打印宽度
options(repr.plot.width = 15, repr.plot.height = 10) # 满幅
p_load(rpart)
p_load(skimr)
s <- skim_tee

In [ ]:
set.seed(943) # for reproducibility
fr <- treemisc::gen_friedman1(1000, nx = 20, sigma = 0.1)
s(fr)

In [ ]:
N <- 10 # 数据规格数
M <- 100 # 每种数据量实验次数
rc <- matrix(-1, M, N) # 用于存储实验结果
colnames(rc) <- as.character(1:N) # 每列对应于不同的数据规格
nr <- nrow(fr)

pb <- txtProgressBar(style = 3)
for (n in 1:N) {
  for (m in 1:M) {
    set.seed(n * 1000 + m)
    I <- sample(nr, (n * nr / N))
    d <- fr[I, ]
    timestart <- Sys.time()
    rpart(y ~ ., data = d, cp = 0)
    rc[m, n] <- difftime(Sys.time(), timestart, units = "secs")
  }
  setTxtProgressBar(pb, n / N)
}

boxplot(rc, xlab = "Each scale unit represents 10% examples", ylab = "time")
lines(colMeans(rc), type = "o", lwd = 2)
grid()

# 可见呈直线，时间复杂度较低